# Import Libraries

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import re

#NLP libraries
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

#Word Vectors
import gensim 
from gensim.models import Word2Vec 
from gensim.models import CoherenceModel

#Visualising Topics
import pyLDAvis
import pyLDAvis.gensim 

C:\Users\lukes\Anaconda3\lib\site-packages\funcy\colls.py:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Set, Iterable, Iterator, defaultdict
C:\Users\lukes\Anaconda3\lib\site-packages\funcy\colls.py:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Set, Iterable, Iterator, defaultdict
C:\Users\lukes\Anaconda3\lib\site-packages\funcy\decorators.py:63: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  spec = inspect.getargspec(func)
C:\Users\lukes\Anaconda3\lib\site-packages\funcy\decorators.py:63: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  spec = inspect.get

# Import Data

In [15]:
data = pd.read_csv('hansard-1979-2018-v261.csv',engine='python')

# Clean Data

### Reset Data

In [16]:
cleaned_data = data
#cleaned_data = pd.read_csv('cleaned_data.csv')

### Drop Columns

In [17]:
cleaned_data = cleaned_data.drop(['pp_id','eo_id','colnum','speech_date','time',
                'url','speaker_id','dods_id','pims_id','ministry',
                'short_list','post_name','person_id','hansard_membership_id',
                'mnis_id','word_count','house_start_date','date_of_birth',
                'house_end_date'],axis=1)

### Remove Speakers

In [18]:
is_not_speaker = cleaned_data['as_speaker']==False

In [19]:
cleaned_data = cleaned_data[is_not_speaker]

### Export Cleaned Data to .csv

In [20]:
#cleaned_data.to_csv('cleaned_data.csv')

### Create Test Dataset

In [21]:
cleaned_data.head()

,speech,year,as_speaker,proper_name,gender,party,party_group,government,age
0,"Mr. Speaker-Elect, it is one of the happiest f...",1979,False,Margaret Thatcher,Female,Conservative,Conservative,True,53.0
1,"Mr. Speaker-Elect, it is my very great privile...",1979,False,James Callaghan,Male,Labour,Labour,False,67.0
2,"On behalf of my right hon. and hon. Friends, M...",1979,False,David Steel,Male,Liberal Democrat,Liberal Democrat,False,41.0
3,"I too, wish to congratulate you, Mr. Speaker-E...",1979,False,James Molyneaux,Male,Ulster Unionist Party,Other,False,58.0
4,"Sometimes, Mr. Speaker-Elect, the House alters...",1979,False,John Parker,Male,Labour,Labour,False,72.0


In [22]:
is_2017 = cleaned_data['year'] ==2017

In [23]:
test_data = cleaned_data[is_2017]

In [24]:
test_data.head()

,speech,year,as_speaker,proper_name,gender,party,party_group,government,age
2145981,I completely agree with my hon. Friend. I have...,2017,False,Damian Green,Male,Conservative,Conservative,True,61.0
2145982,I call Mr Gerald Jones. Where is the fella? He...,2017,False,John Bercow,Male,Conservative,Conservative,True,54.0
2145983,What recent assessment he has made of trends i...,2017,False,Peter Dowd,Male,Labour,Labour,False,59.0
2145984,The Government support those who aspire to be ...,2017,False,Damian Green,Male,Conservative,Conservative,True,61.0
2145985,A Citizens Advice report in August 2015 said t...,2017,False,Peter Dowd,Male,Labour,Labour,False,59.0


# Data Manipulation: Speeches

### Defining Functions to manipulate text

In [25]:
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    new_string = re.sub(r'^https?:\/\/.*[\r\n]*', '', s
                       ) 
    new_string = re.sub('\d+', '', new_string
                       )
    new_string = re.sub('[^\w\s]', ' ', new_string
                       )
    return new_string.lower()

<>:13: DeprecationWarning: invalid escape sequence \d
<>:15: DeprecationWarning: invalid escape sequence \w
<>:13: DeprecationWarning: invalid escape sequence \d
<>:15: DeprecationWarning: invalid escape sequence \w
<>:13: DeprecationWarning: invalid escape sequence \d
<>:15: DeprecationWarning: invalid escape sequence \w
<ipython-input-25-9b2394d5bf9d>:13: DeprecationWarning: invalid escape sequence \d
  new_string = re.sub('\d+', '', new_string
<ipython-input-25-9b2394d5bf9d>:15: DeprecationWarning: invalid escape sequence \w
  new_string = re.sub('[^\w\s]', ' ', new_string


In [26]:
def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    return nltk.word_tokenize(s)

In [27]:
def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    ps = PorterStemmer()
    new_l = ([ps.stem(w) for w in l]
         )

    lemmatizer = WordNetLemmatizer()
    new_l = ([lemmatizer.lemmatize(w) for w in new_l]
            ) 
    
    return new_l

### Clean-up data

In [28]:
test_data['speech'] = test_data['speech'].apply(lambda x: clean_up(x))

C:\Users\lukes\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Tokenize data

In [29]:
test_data['speech'] = test_data['speech'].apply(lambda x: tokenize(x))

C:\Users\lukes\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Stem and Lemmatize data

In [30]:
test_data['speech'] = test_data['speech'].apply(lambda x: stem_and_lemmatize(x))

C:\Users\lukes\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Remove stopwords

In [31]:
stop_words = stopwords.words('english')
test_data['speech'] = test_data['speech'].apply(lambda x: [item for item in x if item not in stop_words])

C:\Users\lukes\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Remove procedural language

In [32]:
proc_lang = ['parliament','govern','opposit','vote','motion','speech','speaker','select','leader','prime',
             'chamber','answer','lord','ladi','back','second','day','view','read','today','n','c','right',
             'hon','member','motion','learn','friend','mr','elect','ha','hi','thi','wa','hous']
for i in test_data['speech']:
    for j in i:
        if j in proc_lang:
            i.remove(j)

In [33]:
test_data.to_csv('test_data.csv')

In [34]:
#test_data = pd.read_csv('test_data.csv')

In [35]:
test_data.head()

,speech,year,as_speaker,proper_name,gender,party,party_group,government,age
2145981,"[complet, agre, friend, alreadi, mention, new,...",2017,False,Damian Green,Male,Conservative,Conservative,True,61.0
2145982,"[call, gerald, jone, fella]",2017,False,John Bercow,Male,Conservative,Conservative,True,54.0
2145983,"[recent, ass, made, trend, level, self, employ]",2017,False,Peter Dowd,Male,Labour,Labour,False,59.0
2145984,"[support, aspir, bos, number, self, employ, pe...",2017,False,Damian Green,Male,Conservative,Conservative,True,61.0
2145985,"[citizen, advic, report, august, said, mani, p...",2017,False,Peter Dowd,Male,Labour,Labour,False,59.0


# Data Manipulation: Bag of Words

In [36]:
all_words = [j for i in test_data['speech'].tolist() for j in i]

In [37]:
all_words = nltk.FreqDist(all_words)

# Word Vectors

In [38]:
model = gensim.models.Word2Vec(test_data['speech'], 
                               min_count = 1,  
                               size = 100, 
                               window = 5) 

In [39]:
dictionary = gensim.corpora.Dictionary(test_data['speech'])

In [40]:
corpus = [dictionary.doc2bow(i) for i in test_data['speech']]

# Running LDA Model

In [94]:
lda_model =  gensim.models.LdaMulticore(corpus, 
                                        num_topics = 8, 
                                        id2word = dictionary,                                    
                                        passes = 10,
                                        workers = 2,
                                       random_state=49)

In [95]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.021*"bill" + 0.017*"amend" + 0.013*"claus" + 0.012*"law" + 0.012*"would" + 0.011*"legisl" + 0.010*"power" + 0.009*"committe" + 0.008*"new" + 0.007*"make"


Topic: 1 
Words: 0.014*"work" + 0.012*"peopl" + 0.008*"constitu" + 0.008*"need" + 0.008*"care" + 0.007*"servic" + 0.007*"support" + 0.006*"mani" + 0.006*"local" + 0.006*"hea"


Topic: 2 
Words: 0.010*"work" + 0.010*"â" + 0.010*"fund" + 0.008*"year" + 0.008*"need" + 0.007*"local" + 0.007*"peopl" + 0.007*"univers" + 0.007*"invest" + 0.007*"new"


Topic: 3 
Words: 0.013*"would" + 0.012*"say" + 0.011*"hon" + 0.011*"veri" + 0.010*"minist" + 0.010*"gentleman" + 0.010*"point" + 0.009*"way" + 0.009*"debat" + 0.009*"peopl"


Topic: 4 
Words: 0.046*"â" + 0.015*"polic" + 0.013*"prison" + 0.012*"report" + 0.011*"offic" + 0.010*"forc" + 0.009*"secretari" + 0.009*"minist" + 0.009*"year" + 0.009*"state"


Topic: 5 
Words: 0.011*"countri" + 0.008*"world" + 0.008*"uk" + 0.008*"intern" + 0.007*"secur" + 0.007*"peopl" + 0.007*"forei

# Extract Dataframe of Topics

In [50]:
for idx, topic in lda_model.print_topics(-1):
    print(idx)

0
1
2
3
4
5
6
7


# Find Dominant Topic in each Speech

In [51]:
def format_topics_sentences(ldamodel=lda_model, bow_corpus=corpus, texts=test_data['speech']):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    return(sent_topics_df)

In [52]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, 
                                                  bow_corpus=corpus, 
                                                  texts=test_data['speech'])

In [53]:
df_topic_sents_keywords.head()

,Dominant_Topic,Perc_Contribution,Topic_Keywords
0,3.0,0.4356,"â, univers, credit, work, peopl, pension, amen..."
1,2.0,0.8247,"hon, debat, minist, gentleman, point, way, giv..."
2,3.0,0.8905,"â, univers, credit, work, peopl, pension, amen..."
3,7.0,0.5681,"â, fund, year, tax, budget, care, need, peopl,..."
4,3.0,0.6912,"â, univers, credit, work, peopl, pension, amen..."


In [54]:
test_data = test_data.reset_index()

In [55]:
final_data = test_data.merge(df_topic_sents_keywords,left_on=test_data.index,right_on=df_topic_sents_keywords.index)

In [56]:
final_data['speeches_count'] = final_data.groupby('proper_name')['proper_name'].transform('count')

In [57]:
final_data.head()

,key_0,index,speech,year,as_speaker,proper_name,gender,party,party_group,government,age,Dominant_Topic,Perc_Contribution,Topic_Keywords,speeches_count
0,0,2145981,"[complet, agre, friend, alreadi, mention, new,...",2017,False,Damian Green,Male,Conservative,Conservative,True,61.0,3.0,0.4356,"â, univers, credit, work, peopl, pension, amen...",359
1,1,2145982,"[call, gerald, jone, fella]",2017,False,John Bercow,Male,Conservative,Conservative,True,54.0,2.0,0.8247,"hon, debat, minist, gentleman, point, way, giv...",2276
2,2,2145983,"[recent, ass, made, trend, level, self, employ]",2017,False,Peter Dowd,Male,Labour,Labour,False,59.0,3.0,0.8905,"â, univers, credit, work, peopl, pension, amen...",192
3,3,2145984,"[support, aspir, bos, number, self, employ, pe...",2017,False,Damian Green,Male,Conservative,Conservative,True,61.0,7.0,0.5681,"â, fund, year, tax, budget, care, need, peopl,...",359
4,4,2145985,"[citizen, advic, report, august, said, mani, p...",2017,False,Peter Dowd,Male,Labour,Labour,False,59.0,3.0,0.6912,"â, univers, credit, work, peopl, pension, amen...",192


In [65]:
model_data = final_data.pivot_table(index='proper_name', columns='Dominant_Topic', aggfunc='count',fill_value=0)

In [66]:
model_data = model_data.iloc[: , [0,1,2,3,4,5,6,7]]

In [67]:
model_data.columns = model_data.columns.droplevel()

In [68]:
model_data = pd.merge(model_data, final_data, how='left', on="proper_name")

In [69]:
grouped_final_data = model_data.drop(['index','speech','key_0','as_speaker','party','Dominant_Topic',
                                      'Perc_Contribution','Topic_Keywords'],axis=1)

In [70]:
grouped_final_data = grouped_final_data.drop_duplicates(subset="proper_name")

In [71]:
grouped_final_data.head()

,proper_name,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,year,gender,party_group,government,age,speeches_count
0,Adam Afriyie,1,2,2,0,1,2,5,3,2017,Male,Conservative,True,51.0,16
16,Adam Holloway,1,1,2,0,0,7,1,0,2017,Male,Conservative,True,52.0,12
28,Adrian Bailey,2,8,0,0,4,1,6,8,2017,Male,Labour,False,71.0,29
57,Afzal Khan,11,9,4,4,1,1,4,3,2017,Male,Labour,False,59.0,37
94,Alan Brown,20,40,26,10,16,19,48,35,2017,Male,SNP,False,46.0,214


# Export Files for Data Visualisation

In [72]:
grouped_final_data.to_csv('grouped_final_data.csv')

In [73]:
final_data.to_csv('final_data.csv')

#### Number of MPs per party

In [78]:
grouped_final_data['party_group'].value_counts()

Conservative        357
Labour              280
SNP                  55
Other                26
Liberal Democrat     17
Name: party_group, dtype: int64

#### Number of speeches per party

In [85]:
final_data['party_group'].value_counts()

Conservative        37710
Labour              16025
SNP                  4258
Other                1573
Liberal Democrat     1186
Name: party_group, dtype: int64

In [102]:
grouped_final_data.groupby(by='party_group').sum().to_csv('grouped_party_data.csv')